# MNIST Classifier from scratch

This is part of the Further Research section in the Chapter 4 of the fastai book. It consists of building a full MNIST Classifier from scratch. 

In [16]:
#hide
# Google Colab Setup
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [17]:
import fastbook
from fastai.vision.all import *
from fastbook import *

## The MNIST Dataset

First, we import the full MNIST Dataset

In [18]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path

In [19]:
print(path.ls(), (path/'training').ls())

[Path('training'), Path('testing')] [Path('training/9'), Path('training/0'), Path('training/7'), Path('training/6'), Path('training/1'), Path('training/8'), Path('training/4'), Path('training/3'), Path('training/2'), Path('training/5')]


Now we want to create a tensor from each image in the dataset, and have them all appropiately labeled. For that reason, we'll create a dictionary and assign each number as a key to a list of the tensors of each class.

In [20]:
digits = {}
for i in range(10):
  digits[i] = (path/'training'/str(i)).ls().sorted()

tensors = [[tensor(Image.open(o)) for o in digits[i]] for i in range(10)]

Some Validations!
- Of the images we indexed

In [21]:
img0_path = digits[2][1040]
im0 = Image.open(img0_path)
im0

- Of the size of each class

In [22]:
for i in range(10):
  print('{}: '.format(i), len(tensors[i]))

0:  5923
1:  6742
2:  5958
3:  6131
4:  5842
5:  5421
6:  5918
7:  6265
8:  5851
9:  5949


- Of the shape of each tensor

In [23]:
tensors[3][0].shape

torch.Size([28, 28])

Now we want to stack each class into a rank-3 tensor:

In [24]:
stacked = [torch.stack(tensors[i]).float()/255 for i in range(10)]
for i in range(10):
  print('{}: '.format(i), stacked[i].shape)

0:  torch.Size([5923, 28, 28])
1:  torch.Size([6742, 28, 28])
2:  torch.Size([5958, 28, 28])
3:  torch.Size([6131, 28, 28])
4:  torch.Size([5842, 28, 28])
5:  torch.Size([5421, 28, 28])
6:  torch.Size([5918, 28, 28])
7:  torch.Size([6265, 28, 28])
8:  torch.Size([5851, 28, 28])
9:  torch.Size([5949, 28, 28])


In [25]:
valid_digits = {}
for i in range(10):
  valid_digits[i] = (path/'testing'/str(i)).ls().sorted()

valid_tensors = [[tensor(Image.open(o)) for o in valid_digits[i]] for i in range(10)]
valid_stacked = [torch.stack(valid_tensors[i]).float()/255 for i in range(10)]

for i in range(10):
  print('valid_{}: '.format(i), valid_stacked[i].shape)

valid_0:  torch.Size([980, 28, 28])
valid_1:  torch.Size([1135, 28, 28])
valid_2:  torch.Size([1032, 28, 28])
valid_3:  torch.Size([1010, 28, 28])
valid_4:  torch.Size([982, 28, 28])
valid_5:  torch.Size([892, 28, 28])
valid_6:  torch.Size([958, 28, 28])
valid_7:  torch.Size([1028, 28, 28])
valid_8:  torch.Size([974, 28, 28])
valid_9:  torch.Size([1009, 28, 28])


Finally, for our `x` inputs, we want to create a single rank-2 tensor containing all classes stacked and every image transformed to a vector. For the `y` targets, we want a hot-encoded vector for each integer the image represents.

In [26]:
train_x = torch.cat([stacked[i] for i in range(10)]).view(-1,28*28)
train_y = []
for i in range(10):
    zeros = [0]*tensor(10)
    zeros[i] = 1.
    train_y += zeros * len(digits[i])
train_y = tensor(train_y).view((-1,10))
train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 10]))

In [27]:
dset = list(zip(train_x, train_y))

Doing the same for the vaildation set

In [28]:
valid_x = torch.cat([valid_stacked[i] for i in range(10)]).view(-1,28*28)
valid_y = []
for i in range(10):
    zeros = [0]*tensor(10)
    zeros[i] = 1.
    valid_y += zeros * len(valid_digits[i])
valid_y = tensor(valid_y).view((-1,10))
valid_x.shape, valid_y.shape

(torch.Size([10000, 784]), torch.Size([10000, 10]))

In [29]:
x,y = dset[9]; x.shape, y

(torch.Size([784]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [30]:
valid_dset = list(zip(valid_x, valid_y))

## Loss function

First, we define a function to initialize our parameters. It is important to remember we have 10 classes!

In [31]:
def init_params(size, std=1): return (torch.randn(size)*std).requires_grad_()
weights = init_params((28*28,10))
bias = init_params(1)

Now, we want to build a libear function for our predictions. We are using a softmax function, since we want our predictions to output a probability for each one of the 10 classes.

In [32]:
def linear1(xb): return F.softmax(xb@weights + bias, dim=1)
preds = linear1(train_x)
preds

tensor([[2.7995e-07, 4.1746e-16, 6.6198e-08,  ..., 4.7592e-17, 3.4086e-05, 9.5948e-14],
        [4.6760e-10, 7.4733e-15, 3.9889e-16,  ..., 9.7262e-16, 2.8436e-10, 2.4661e-11],
        [1.2541e-18, 9.8313e-26, 2.3247e-14,  ..., 9.7353e-26, 7.0008e-08, 3.5697e-21],
        ...,
        [1.6117e-12, 1.0476e-11, 9.3556e-07,  ..., 2.5763e-08, 6.4884e-06, 1.5282e-04],
        [1.3881e-11, 8.2489e-09, 2.4626e-11,  ..., 5.8326e-06, 2.7856e-11, 1.1645e-09],
        [2.0035e-10, 9.7288e-12, 1.2878e-11,  ..., 3.5700e-09, 8.0692e-05, 1.6643e-05]], grad_fn=<SoftmaxBackward0>)

Now we build a loss function

In [33]:
def mnist_loss(predictions, targets):
  predictions = predictions.softmax(1)
  return torch.where(targets==1,1-predictions,predictions).mean()

And define our `DataLoaders`

In [34]:
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size = 256)
xb, yb = first(dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256, 10]))

### Sidebar: Tests

In [35]:
#hide
## Creating a mini-batch of size 5 for testing
batch = train_x[:5]
batch.shape

torch.Size([5, 784])

In [36]:
preds = linear1(batch)
loss = mnist_loss(preds, train_y[:5])
loss

tensor(0.1829, grad_fn=<MeanBackward0>)

In [37]:
# Calculating the gradients
loss.backward()
weights.grad.shape,weights.grad.mean(),bias.grad

(torch.Size([784, 10]), tensor(4.1818e-12), tensor([2.2697e-10]))

### End sidebar

Now we define a function that calculates the gradients of our parameters

In [38]:
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  loss.backward()

And a function to train each epoch, and one to calculate the accuracy for each batch

In [39]:
def train_epoch(model, lr, params):
  for xb,yb in dl:
    calc_grad(xb, yb, model)
    for p in params:
      p.data -= p.grad*lr 
      p.grad.zero_()

def batch_accuracy(xb, yb):
    preds = xb.argmax(dim=1)
    true_classes = yb.argmax(dim=1)
    correct = (preds == true_classes).sum().item()
    total = xb.size(0)
    return tensor(correct / total)

In [40]:
# Making sure it works
batch_accuracy(linear1(batch), train_y[:5])

tensor(0.)

Now we create a function to calculate the accuracy for each epoch

In [43]:
def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
  return round(torch.stack(accs).mean().item(),4)
validate_epoch(linear1)

0.0894

In [44]:
# One example 
lr = 1.
params = weights,bias
train_epoch(linear1, lr, params)
validate_epoch(linear1)

0.0917

Training for more epochs

In [30]:
for i in range(20):
  train_epoch(linear1, lr, params)
  print(validate_epoch(linear1), end=' ')

0.1467 0.1561 0.1688 0.1811 0.1923 0.2051 0.2201 0.2297 0.2393 0.2501 0.2602 0.2716 0.2823 0.2925 0.3037 0.3123 0.3205 0.3284 0.3359 0.3418 

As we can see, accuracy is improving, which means the model is learning! However, we want to improve its performance.

## Creating an Optimizer

We can create the same linear function we just built using PyTorc's `nn.Linear()` module.

In [45]:
linear_model = nn.Linear(28*28,10)
w,b = linear_model.parameters()
w.shape,b.shape

(torch.Size([10, 784]), torch.Size([10]))

Now, we create an optimizer and a function to train one epoch using it

In [46]:
class BasicOptim:
  def __init__(self,params,lr): self.params,self.lr = list(params), lr
  
  def step(self, *args, **kwargs):
    for p in self.params: p.data -= p.grad.data * self.lr

  def zero_grad(self, *args, **kwargs):
    for p in self.params: p.grad = None

opt = BasicOptim(linear_model.parameters(), lr)

def train_epoch(model):
  for xb,yb in dl:
    calc_grad(xb,yb,model)
    opt.step()
    opt.zero_grad()

In [47]:
validate_epoch(linear_model)

0.1067

Now we can train the model!

In [48]:
def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [49]:
train_model(linear_model, 20)

0.1403 0.3382 0.2899 0.338 0.3849 0.4236 0.4773 0.5131 0.5325 0.552 0.5722 0.5906 0.6071 0.6213 0.6332 0.6451 0.6551 0.6643 0.6736 0.6809 

Using the `DataLoaders` class and building a `Learner` with the functions we defined before

In [50]:
dls = DataLoaders(dl, valid_dl)
learn = Learner(dls, nn.Linear(28*28,10), opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [51]:
learn.fit(20,lr=lr)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.157329,0.167066,0.159700,00:00
1,0.105744,0.152815,0.245000,00:00
2,0.068694,0.137599,0.334900,00:00
3,0.044959,0.120876,0.430500,00:00
4,0.038344,0.108376,0.493700,00:00
5,0.035399,0.100201,0.531700,00:00
6,0.033736,0.094040,0.558900,00:00
7,0.032681,0.088980,0.582200,00:00
8,0.031943,0.084674,0.600900,00:00
9,0.031394,0.080926,0.618600,00:00


As we can see, accuracy is improving too. However, we can train a better model if we add a nonlinearity between our linear functions

## Adding a non-linearity

As we have 10 classes, the output layer must output 10 values as well

In [55]:
simple_net = nn.Sequential(
    nn.Linear(28*28,300),
    nn.ReLU(),
    nn.Linear(300,10)
)

 Now we build a learner for this simple net

In [56]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [57]:
learn.fit(40,0.1)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.181721,0.177023,0.440900,00:00
1,0.180410,0.170850,0.480100,00:00
2,0.176802,0.161541,0.441400,00:00
3,0.163254,0.150322,0.347700,00:00
4,0.136698,0.144714,0.323600,00:00
5,0.122496,0.131697,0.387900,00:00
6,0.118309,0.120395,0.445700,00:00
7,0.118121,0.111767,0.470900,00:00
8,0.114683,0.105283,0.492100,00:00
9,0.105525,0.100959,0.513400,00:00


Accuracy improved! Now we understand how to build a Learner from scratch. We can leverage fastai's library to build a more robust model with just a couple lines of code!

## More robust model

We will use the `ImageDataLoaders` class, and set the device to 'mps' to leverage the power of the Apple Sillicon GPU.

In [58]:
dls = ImageDataLoaders.from_folder(path, train='training', valid='testing', device='mps')
learn = vision_learner(dls, resnet18, pretrained=False, loss_func=F.cross_entropy, metrics=accuracy)

In [60]:
learn.fit_one_cycle(1,0.1)

epoch,train_loss,valid_loss,accuracy,time
0,nan,0.045411,0.988100,01:08


As we can see, the resnet18 neural net achieves >98% accuracy in just one epoch!